In [1]:
import os
import sqlite3
from pathlib import Path
import pandas as pd
import shutil

#gather data from a specific csv file and return as a pandas df
BASE = Path.cwd()
DATA = BASE / "training_data"
#Make sure training_data folder exists
DATA.mkdir(parents= True, exist_ok= True)

In [16]:
#Get the user path 
user_path = os.path.expanduser("~")
#Make the path to the kaggle json
downloads_path = os.path.join(user_path, 'Downloads/kaggle.json')

#Make the .kaggle folder in user
kaggle_folder = os.path.join(user_path, ".kaggle")
os.makedirs(kaggle_folder, exist_ok= True)

print(os.path.exists(downloads_path))

if os.path.exists(downloads_path):
    try:
        print("moving kaggle.json to user")
        #Make .kaggle folder path and check it exists
        
        #Move json to the folder
        shutil.move(downloads_path, kaggle_folder)
    except Exception as e:
        print(f"Error when trying to move kaggle.json: {e}")
else:
    #attempt to check if kaggle.json already exists
    final = os.path.join(kaggle_folder, "kaggle.json")
    if os.path.exists(final):
        print("kaggle json already exists in user")
    else:
        print("please download kaggle.json")

#print("Successfully moved json file")

False
kaggle json already exists in user


In [17]:
#TO USE THIS API you must have a .kaggle folder in your 'C:\NAME' directory -> then paste the kaggle.json authenticator
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print(api.dataset_list_files('wordsforthewise/lending-club').files)

api.dataset_download_files('wordsforthewise/lending-club', path= DATA, unzip=True)

[{"ref": "", "datasetRef": null, "ownerRef": null, "name": "accepted_2007_to_2018Q4.csv.gz", "creationDate": "2019-12-17T23:30:10.124Z", "description": null, "fileType": null, "url": null, "totalBytes": 392582231, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv", "creationDate": "2019-04-10T18:09:09.272Z", "description": null, "fileType": null, "url": null, "totalBytes": 1675133810, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "rejected_2007_to_2018Q4.csv.gz", "creationDate": "2019-12-17T23:30:04.004Z", "description": null, "fileType": null, "url": null, "totalBytes": 255470782, "columns": []}, {"ref": "", "datasetRef": null, "ownerRef": null, "name": "rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv", "creationDate": "2019-04-10T18:09:22.367Z", "description": null, "fileType": null, "url": null, "totalBytes": 1782281620, "columns": []}]
Dataset URL: https://www.kaggle

In [18]:
def retrieve_training_csv(): 
    """ Function that returns all the csv files in the training data folder as a dataframe object """ 
    csv_list = list(DATA.glob("**/*.csv")) 
    return_list = [] 
    print(csv_list)

    #traverse each item inthe data path, and if a file ending in .csv is found, turn it into a df and append to return list 
    for item in csv_list: 
        if os.path.isfile(item): 
            print(f"{item} is a file") 
            return_list.append(pd.read_csv(item)) 
        else: print(f"{item} is folder or dir")

    if return_list: 
        return return_list 
    else: 
        print(f"No csv files found in {DATA}") 
        return []

def get_dir_size(path): 
    """ Get directory size in MBs """ 
    total = 0 
    for dirpath, dirs, files in os.walk(path):
        for f in files:
            file_path = os.path.join(dirpath, f)
            
            try:
                total += os.path.getsize(file_path) 
                print(f"{total/1000000} MB")
            except: 
                continue 
        return total/1000000
    
def delete_large_files(): 
    """ Function that deletes the large files downloaded from kaggle to save space 
    Deletes files to prevent any storage errors when pushing code to github """

    for paths in DATA.glob("**/*"): 
        if os.path.isdir(paths): 
            try:
                size = get_dir_size(paths) 
                print(f"size of {paths} is {size}")

                if size > 100: 
                    print(f"deleting {paths}") 
                    shutil.rmtree(paths) 

            except Exception as e:
                print(f"Error when trying to get size or delete file: {e}")
        else:
            if paths.suffix == ".gz":
                os.remove(paths)
                #print(paths)
        

df_list = retrieve_training_csv()

[WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv'), WindowsPath('c:/Users/Saul/Desktop/Credit Risk Assessment/credit-risk-assessment/ETL/training_data/workspace/loan_data.csv')]
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\accepted_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv is folder or dir
c:\Users\Saul\Desktop\Credit Risk A

C:\Users\Saul\AppData\Local\Temp\ipykernel_22428\3710330960.py:11: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  return_list.append(pd.read_csv(item))


c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\rejected_2007_to_2018q4.csv\rejected_2007_to_2018Q4.csv is a file
c:\Users\Saul\Desktop\Credit Risk Assessment\credit-risk-assessment\ETL\training_data\workspace\loan_data.csv is a file


In [19]:
#create different pointers to the csvs in the list for easier access
raw_accepted_loans = df_list[0]
raw_rejected_loans = df_list[1]
raw_loans_paid_info = df_list[2]

In [20]:
raw_accepted_loans.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
5,68426831,NaN,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
6,68476668,NaN,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
7,67275481,NaN,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
8,68466926,NaN,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
9,68616873,NaN,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
al_columns = list(raw_accepted_loans.columns)
print(al_columns)
raw_accepted_loans['loan_status'].head(10)
num_cols = [
    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",
    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",
    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"
]

text_cols = [
    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"
]


"""
raw_specific_al = raw_accepted_loans[target_columns]
raw_specific_al.head(10).T
"""

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq',

'\nraw_specific_al = raw_accepted_loans[target_columns]\nraw_specific_al.head(10).T\n'

**Clean Accepted Loans Data**
- Provide a data summary
- Standardize all data in the columns to reasonable types (ex: object -> float)
- Clean up any bad data (missing values, duplicates, etc)


In [22]:
#only find columns with valid ids (not string or empty)
cleaned_accepted_loans = raw_accepted_loans[num_cols + text_cols]
cleaned_accepted_loans = cleaned_accepted_loans[pd.to_numeric(cleaned_accepted_loans['id'], errors='coerce').notna()]

#convert id column to int instead of obj
cleaned_accepted_loans['id'] = cleaned_accepted_loans['id'].astype('int64')

In [23]:
#fix the the term column to make it float (and in months)
cleaned_accepted_loans.rename(columns = {'term': 'term_months'}, inplace = True)
num_cols[3] = 'term_months'
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].str.replace(" months", "")
cleaned_accepted_loans['term_months'] = cleaned_accepted_loans['term_months'].astype('int64')

In [24]:
cleaned_accepted_loans.isna().sum()

id                         0
loan_amnt                  0
funded_amnt                0
term_months                0
int_rate                   0
installment                0
annual_inc                 4
dti                     1711
delinq_2yrs               29
fico_range_low             0
fico_range_high            0
inq_last_6mths            30
open_acc                  29
revol_bal                  0
revol_util              1802
total_acc                 29
pub_rec_bankruptcies    1365
home_ownership             0
loan_status                0
purpose                    0
application_type           0
verification_status        0
dtype: int64

In [25]:
#find any na rows and fill them as needed
for rows in num_cols:
    cleaned_accepted_loans.loc[:, rows] = cleaned_accepted_loans[rows].fillna(cleaned_accepted_loans[rows].median())

cleaned_accepted_loans.isna().sum()


id                      0
loan_amnt               0
funded_amnt             0
term_months             0
int_rate                0
installment             0
annual_inc              0
dti                     0
delinq_2yrs             0
fico_range_low          0
fico_range_high         0
inq_last_6mths          0
open_acc                0
revol_bal               0
revol_util              0
total_acc               0
pub_rec_bankruptcies    0
home_ownership          0
loan_status             0
purpose                 0
application_type        0
verification_status     0
dtype: int64

In [26]:
#Get a sample of the cleaned up 
sampled_accepted_loans = cleaned_accepted_loans.sample(200000, random_state = 821)
sampled_accepted_loans

,id,loan_amnt,funded_amnt,term_months,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,open_acc,revol_bal,revol_util,total_acc,pub_rec_bankruptcies,home_ownership,loan_status,purpose,application_type,verification_status
1488822,135703522,6500.0,6500.0,36,20.39,242.86,47000.0,10.29,0.0,660.0,...,11.0,5856.0,43.1,27.0,1.0,MORTGAGE,Current,house,Individual,Not Verified
567573,117605717,20000.0,20000.0,36,9.44,640.10,156000.0,26.65,0.0,740.0,...,20.0,20106.0,40.9,32.0,0.0,MORTGAGE,Current,home_improvement,Individual,Not Verified
1933073,1158084,26400.0,26400.0,60,9.76,557.81,90000.0,18.91,0.0,770.0,...,9.0,13856.0,20.6,24.0,0.0,MORTGAGE,Fully Paid,debt_consolidation,Individual,Verified
539548,119384164,8000.0,8000.0,36,16.02,281.34,38000.0,8.28,0.0,660.0,...,3.0,3399.0,75.5,12.0,0.0,OWN,Current,major_purchase,Individual,Source Verified
1057196,71806132,28000.0,28000.0,60,26.57,847.81,120000.0,24.77,0.0,680.0,...,12.0,65167.0,76.5,28.0,0.0,MORTGAGE,Charged Off,debt_consolidation,Individual,Source Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957236,105781229,18500.0,18500.0,36,6.99,571.15,81528.0,13.90,0.0,750.0,...,13.0,16760.0,36.0,26.0,0.0,MORTGAGE,Current,debt_consolidation,Individual,Source Verified
340434,43659370,20800.0,20800.0,60,10.99,452.14,91248.0,23.19,0.0,740.0,...,19.0,32518.0,33.7,46.0,0.0,MORTGAGE,Fully Paid,debt_consolidation,Individual,Source Verified
52235,63215769,12000.0,12000.0,36,7.89,375.43,70000.0,22.65,0.0,705.0,...,19.0,17132.0,58.7,39.0,0.0,RENT,Fully Paid,credit_card,Individual,Source Verified
2119521,121932789,25000.0,25000.0,60,11.99,555.99,81000.0,10.92,0.0,705.0,...,13.0,13651.0,41.9,22.0,0.0,RENT,Current,debt_consolidation,Joint App,Not Verified


**Clean Rejected Loan Data**

In [27]:
cleaned_rejected_loans = raw_rejected_loans.copy()
cleaned_rejected_loans.isna().sum()

Amount Requested               0
Application Date               0
Loan Title                  1305
Risk_Score              18497630
Debt-To-Income Ratio           0
Zip Code                     293
State                         22
Employment Length         951355
Policy Code                  918
dtype: int64

In [28]:
#Drop unneeded columns 
cleaned_rejected_loans = cleaned_rejected_loans.drop(columns=['Risk_Score', 'Zip Code', 'State', 'Policy Code', 'Employment Length'])
rl_cols = list(cleaned_rejected_loans.columns)
print(rl_cols)

cleaned_rejected_loans.dtypes

['Amount Requested', 'Application Date', 'Loan Title', 'Debt-To-Income Ratio']


Amount Requested        float64
Application Date         object
Loan Title               object
Debt-To-Income Ratio     object
dtype: object

In [29]:
print(type(cleaned_rejected_loans['Amount Requested'].dtype))

<class 'numpy.dtypes.Float64DType'>


In [30]:
#Fix any missing values in the rejection loans
for rows in rl_cols:
    #fix the rows that are of float type
    if cleaned_rejected_loans[rows].dtypes == 'float64':
        print(f"replacing {rows} with median float")
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(cleaned_rejected_loans[rows].median())
    else:
        #get the highest repeated string and replace N/A's with that string
        print(f"replacing {rows} na's with most repeated string")
        replacement_string = cleaned_rejected_loans[rows].value_counts().reset_index().at[0,rows]
        cleaned_rejected_loans.loc[:, rows] = cleaned_rejected_loans[rows].fillna(replacement_string)

replacing Amount Requested with median float
replacing Application Date na's with most repeated string
replacing Loan Title na's with most repeated string
replacing Debt-To-Income Ratio na's with most repeated string


In [33]:
cleaned_rejected_loans.isna().sum()
print(len(cleaned_rejected_loans))

cleaned_rejected_loans.head(10)

27648741


,Amount Requested,Application Date,Loan Title,Debt-To-Income Ratio
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,10%
1,1000.0,2007-05-26,Consolidating Debt,10%
2,11000.0,2007-05-27,Want to consolidate my debt,10%
3,6000.0,2007-05-27,waksman,38.64%
4,1500.0,2007-05-27,mdrigo,9.43%
5,15000.0,2007-05-27,Trinfiniti,0%
6,10000.0,2007-05-27,NOTIFYi Inc,10%
7,3900.0,2007-05-27,For Justin.,10%
8,3000.0,2007-05-28,title?,10%
9,2500.0,2007-05-28,timgerst,11.76%


In [2]:
#Get csv of accepted loans from 2023 HMDA
import requests
try:
    #response = requests.get("https://ffiec.cfpb.gov/v2/data-browser-api/view/nationwide/csv?years=2023&actions_taken=1&loan_purpose=1")
    accepted_response = requests.get("https://files.ffiec.cfpb.gov/data-browser/datasets/2023/filtered-queries/one-year/a5a77b5e9528ccb95aae0dc60cea9d70.csv", stream= True)
    accepted_response.raise_for_status()
except requests.HTTPError as e:
    print(f"Error getting HDMI data: {e}")

In [ ]:
#turning response into csv
import io
import csv
raw_string = accepted_response.text
split_string = raw_string.split('\n')
#write to a csv in the training data folder
string_columns = split_string[0].split(',')
string_data = [rows.split(',') for rows in split_string[1:]]

hdma_path = DATA / 'hdma_accepted_raw.csv'

with open(hdma_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(string_columns)
    csv_writer.writerows(string_data)

In [3]:
import io
import csv
hdma_path = DATA / 'hdma_accepted_raw.csv'
space_saver = pd.read_csv(hdma_path, low_memory= False)

In [4]:
hdma_accepted_parquet = DATA / 'hdma_accepted_raw.parquet.gzip'
#space_saver.to_parquet(hdma_accepted_parquet)
for columns in space_saver.columns:
    if space_saver[columns].dtype == 'object':
        print('object')
        space_saver[columns] = space_saver[columns].astype('string')
    else:
        print(space_saver[columns].dtype)

float64
object
float64
object
float64
float64
object
object
object
object
object
object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
object
object
float64
object
object
object
object
object
object
object
object
float64
float64
float64
float64
object
float64
float64
float64
float64
object
object
float64
object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
object
object
object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [ ]:
#columns that do not affect model predictions in any meaningful way. dropped in order to save space when uploading to repository
drop_cols = [
    'tract_population', 'tract_minority_population_percent', 'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
    'tract_owner_occupied_units', 'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units', 'aus-1', 'aus-2', 'aus-3', 'aus-4',
    'aus-5', 'applicant_ethnicity-1', 'applicant_ethnicity-2', 'applicant_ethnicity-3', 'applicant_ethnicity-4', 'applicant_ethnicity-5',
    'co-applicant_ethnicity-1', 'co-applicant_ethnicity-2', 'co-applicant_ethnicity-3', 'co-applicant_ethnicity-4', 'co-applicant_ethnicity-5',
    'applicant_ethnicity_observed', 'co-applicant_ethnicity_observed', 'applicant_race-1', 'applicant_race-2', 'applicant_race-3', 'applicant_race-4',
    'applicant_race-5', 'co-applicant_race-1', 'co-applicant_race-2', 'co-applicant_race-3', 'co-applicant_race-4', 'co-applicant_race-5',
    'applicant_race_observed', 'co-applicant_race_observed', 'applicant_sex', 'co-applicant_sex', 'applicant_sex_observed', 'co-applicant_sex_observed',
    'applicant_age', 'co-applicant_age', 'applicant_age_above_62', 'co-applicant_age_above_62'
    ]

In [25]:
space_saver.drop(columns = drop_cols).to_parquet(hdma_accepted_parquet, compression='gzip')

In [26]:
recovered = pd.read_parquet(hdma_accepted_parquet, columns=temp_num_cols + temp_text_cols)

#print(temp_num_cols + temp_text_cols)

In [27]:
recovered.head(10)

,activity_year,action_taken,preapproval,loan_purpose,loan_amount,loan_to_value_ratio,interest_rate,total_loan_costs,loan_term,income,debt_to_income_ratio,derived_loan_product_type,applicant_credit_score_type,co-applicant_credit_score_type,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4
0,2023.0,1.0,2.0,1.0,75000.0,80.00000,7.500,<NA>,360,180.0,44,Conventional:First Lien,3.0,10.0,10.0,NaN,NaN,NaN
1,2023.0,1.0,2.0,1.0,445000.0,84.00000,6.575,12338.70,360,153.0,37,Conventional:First Lien,1.0,9.0,10.0,NaN,NaN,NaN
2,2023.0,1.0,2.0,1.0,145000.0,96.50000,7.250,8195.48,360,73.0,42,FHA:First Lien,2.0,10.0,10.0,NaN,NaN,NaN
3,2023.0,1.0,2.0,1.0,425000.0,75.00000,7.500,<NA>,360,NaN,46,Conventional:First Lien,2.0,9.0,10.0,NaN,NaN,NaN
4,2023.0,1.0,2.0,1.0,425000.0,100.88000,7.000,8671.70,360,84.0,49,Conventional:First Lien,2.0,9.0,10.0,NaN,NaN,NaN
5,2023.0,1.0,2.0,1.0,325000.0,80.00000,7.000,6683.09,360,189.0,30%-<36%,Conventional:First Lien,3.0,10.0,10.0,NaN,NaN,NaN
6,2023.0,1.0,2.0,1.0,305000.0,95.00000,6.500,16722.90,360,65.0,46,FHA:First Lien,3.0,10.0,10.0,NaN,NaN,NaN
7,2023.0,1.0,1.0,1.0,175000.0,100.00000,7.375,10773.59,360,61.0,44,FHA:First Lien,1.0,10.0,10.0,NaN,NaN,NaN
8,2023.0,1.0,2.0,1.0,125000.0,50.00000,6.990,5073.25,360,132.0,20%-<30%,Conventional:First Lien,3.0,9.0,10.0,NaN,NaN,NaN
9,2023.0,1.0,2.0,1.0,275000.0,87.86900,6.750,10391.26,360,100.0,39,FHA:First Lien,3.0,9.0,10.0,NaN,NaN,NaN


In [ ]:
remove_index = []

#get the column numbers needed
for columns in temp_num_cols:
    remove_index.append(string_columns.index(columns))

for columns in temp_text_cols:
    remove_index.append(string_columns.index(columns))

print(remove_index)

[0, 12, 14, 16, 21, 22, 23, 26, 31, 45, 46, 7, 47, 48, 88, 89, 90, 91]


In [ ]:
finalized_cols = [row for row in string_columns if string_columns.index(row) in remove_index]
finalized_text = [row for row in string_data if string_data.index(row) in remove_index]
print((finalized_cols))
print(len(string_data))

: 

In [13]:
temp_list = [i for i in range(0,100) if i not in remove_index]
print(temp_list)
split_string.clear()
print(len(temp_list))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 18, 19, 20, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
89


In [4]:
print(string_columns)
print(string_data[0])
hmda_accepted = pd.DataFrame(string_data, columns=string_columns)


['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open-end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'income', 'debt_to_income_ratio', 'applicant_credit_score

In [2]:
#https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/lar-data-fields#loan_amount
temp_num_cols = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_to_value_ratio',
    'loan_term', 'income', 'debt_to_income_ratio'
    ]
temp_text_cols = [
    'derived_loan_product_type', 'applicant_credit_score_type', 'co-applicant_credit_score_type', 'denial_reason-1'
    ]

"""
num_cols = [
    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",
    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",
    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"
]

text_cols = [
    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"
]
"""

'\nnum_cols = [\n    "id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "annual_inc",\n    "dti", "delinq_2yrs", "fico_range_low", "fico_range_high", "inq_last_6mths",\n    "open_acc", "revol_bal", "revol_util", "total_acc", "pub_rec_bankruptcies"\n]\n\ntext_cols = [\n    "home_ownership", "loan_status", "purpose", "application_type", "verification_status"\n]\n'

In [71]:
hmda_accepted.iloc[0:5,20:30]

,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points
0,1,75000.0,80.00000,7.500,NA,3,NA,NA,NA,NA
1,2,445000.0,84.00000,6.575,0.122,2,12338.70,NA,7963.00,6615.00
2,2,145000.0,96.50000,7.250,1.598,2,8195.48,NA,3429.80,1739.80
3,1,425000.0,75.00000,7.500,NA,3,NA,NA,NA,NA
4,2,425000.0,100.88000,7.000,0.857,2,8671.70,NA,6617.70,


In [8]:
hmda_accepted = hmda_accepted[temp_num_cols + temp_text_cols]
hmda_accepted.columns

Index(['activity_year', 'action_taken', 'preapproval', 'loan_purpose',
       'loan_amount', 'loan_to_value_ratio', 'interest_rate',
       'total_loan_costs', 'loan_term', 'income', 'debt_to_income_ratio',
       'derived_loan_product_type', 'applicant_credit_score_type',
       'co-applicant_credit_score_type', 'denial_reason-1', 'denial_reason-2',
       'denial_reason-3', 'denial_reason-4'],
      dtype='object')

In [9]:
hmda_accepted.head(10)

,activity_year,action_taken,preapproval,loan_purpose,loan_amount,loan_to_value_ratio,interest_rate,total_loan_costs,loan_term,income,debt_to_income_ratio,derived_loan_product_type,applicant_credit_score_type,co-applicant_credit_score_type,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4
0,2023,1,2,1,75000.0,80.00000,7.500,NA,360,180,44,Conventional:First Lien,3,10,10,,,
1,2023,1,2,1,445000.0,84.00000,6.575,12338.70,360,153,37,Conventional:First Lien,1,9,10,,,
2,2023,1,2,1,145000.0,96.50000,7.250,8195.48,360,73,42,FHA:First Lien,2,10,10,,,
3,2023,1,2,1,425000.0,75.00000,7.500,NA,360,NA,46,Conventional:First Lien,2,9,10,,,
4,2023,1,2,1,425000.0,100.88000,7.000,8671.70,360,84,49,Conventional:First Lien,2,9,10,,,
5,2023,1,2,1,325000.0,80.00000,7.000,6683.09,360,189,30%-<36%,Conventional:First Lien,3,10,10,,,
6,2023,1,2,1,305000.0,95.00000,6.500,16722.90,360,65,46,FHA:First Lien,3,10,10,,,
7,2023,1,1,1,175000.0,100.00000,7.375,10773.59,360,61,44,FHA:First Lien,1,10,10,,,
8,2023,1,2,1,125000.0,50.00000,6.990,5073.25,360,132,20%-<30%,Conventional:First Lien,3,9,10,,,
9,2023,1,2,1,275000.0,87.86900,6.750,10391.26,360,100,39,FHA:First Lien,3,9,10,,,


In [15]:
#Get csv of rejected loans from 2023 HMDA
import requests
try:
    rejected_response = requests.get("https://ffiec.cfpb.gov/v2/data-browser-api/view/nationwide/csv?years=2023&actions_taken=3&loan_purpose=1")
    rejected_response.raise_for_status()
except requests.HTTPError as e:
    print(f"Error getting HDMI data: {e}")

In [ ]:
rejected_string = rejected_response.text
split_string = rejected_string.split('\n')
#write to a csv in the training data folder
rejected_columns = split_string[0].split(',')
rejected_data = [rows.split(',') for rows in split_string[1:]]

['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open-end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'income', 'debt_to_income_ratio', 'applicant_credit_score

In [20]:
print(len(rejected_columns))
print(rejected_data[0])
hdma_rejected_path = DATA / 'hdma_rejected_raw.csv'

99
['2023', '549300VZVN841I2ILS84', '35004', 'NY', '36059', '36059410400', 'C', 'Conventional:First Lien', 'Single Family (1-4 Units):Site-Built', 'Not Hispanic or Latino', 'Asian', 'Male', '3', '0', '2', '1', '1', '1', '2', '2', '2', '545000.0', '80.00000', 'NA', 'NA', '3', 'NA', 'NA', 'NA', 'NA', 'NA', '360', 'NA', 'NA', '2', '2', '2', '2', '675000', '1', '1', '3', '5', '1', 'NA', '110', '50%-60%', '2', '9', '2', '', '', '', '', '2', '', '', '', '', '2', '2', '2', '', '', '', '', '2', '', '', '', '', '2', '2', '1', '1', '2', '2', '25-34', '55-64', 'No', 'No', '1', '1', '2', '', '', '', '', '1', '', '', '', '5647', '88.15', '156300', '92.78', '1167', '1428', '68']


In [33]:
#Write raw data to unique csv
import csv
with open(hdma_rejected_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(rejected_columns)
    csv_writer.writerows(rejected_data)

In [56]:
#Temporarily read csv, drop unneeded columns, and parquet the file to save space
rejected_temp = pd.read_csv(hdma_rejected_path, low_memory= False)

hdma_rejected_parquet = DATA / 'hdma_rejected_raw.parquet.gzip'
#space_saver.to_parquet(hdma_accepted_parquet)
for columns in rejected_temp.columns:
    if rejected_temp[columns].dtype == 'object':
        print('object')
        rejected_temp[columns] = rejected_temp[columns].astype('string')

NameError: name 'hdma_rejected_path' is not defined

In [35]:
rejected_temp.drop(columns = drop_cols).to_parquet(hdma_rejected_parquet, compression='gzip')

In [3]:
hdma_accepted_parquet = DATA / 'hdma_accepted_raw.parquet.gzip'
hdma_rejected_parquet = DATA / 'hdma_rejected_raw.parquet.gzip'
accepted_recovered = pd.read_parquet(hdma_accepted_parquet, columns=temp_num_cols + temp_text_cols)
rejected_recovered = pd.read_parquet(hdma_rejected_parquet, columns=temp_num_cols + temp_text_cols)

In [58]:
accepted_recovered.isna().sum()
accepted_recovered.head(10)

,activity_year,action_taken,preapproval,loan_purpose,loan_amount,loan_to_value_ratio,loan_term,income,debt_to_income_ratio,derived_loan_product_type,applicant_credit_score_type,co-applicant_credit_score_type,denial_reason-1
0,2023.0,1.0,2.0,1.0,75000.0,80.00000,360,180.0,44,Conventional:First Lien,3.0,10.0,10.0
1,2023.0,1.0,2.0,1.0,445000.0,84.00000,360,153.0,37,Conventional:First Lien,1.0,9.0,10.0
2,2023.0,1.0,2.0,1.0,145000.0,96.50000,360,73.0,42,FHA:First Lien,2.0,10.0,10.0
3,2023.0,1.0,2.0,1.0,425000.0,75.00000,360,NaN,46,Conventional:First Lien,2.0,9.0,10.0
4,2023.0,1.0,2.0,1.0,425000.0,100.88000,360,84.0,49,Conventional:First Lien,2.0,9.0,10.0
5,2023.0,1.0,2.0,1.0,325000.0,80.00000,360,189.0,30%-<36%,Conventional:First Lien,3.0,10.0,10.0
6,2023.0,1.0,2.0,1.0,305000.0,95.00000,360,65.0,46,FHA:First Lien,3.0,10.0,10.0
7,2023.0,1.0,1.0,1.0,175000.0,100.00000,360,61.0,44,FHA:First Lien,1.0,10.0,10.0
8,2023.0,1.0,2.0,1.0,125000.0,50.00000,360,132.0,20%-<30%,Conventional:First Lien,3.0,9.0,10.0
9,2023.0,1.0,2.0,1.0,275000.0,87.86900,360,100.0,39,FHA:First Lien,3.0,9.0,10.0


In [59]:
rejected_recovered.isna().sum()
rejected_recovered.head(10)
#rejected_recovered.dtypes

,activity_year,action_taken,preapproval,loan_purpose,loan_amount,loan_to_value_ratio,loan_term,income,debt_to_income_ratio,derived_loan_product_type,applicant_credit_score_type,co-applicant_credit_score_type,denial_reason-1
0,2023.0,3.0,2.0,1.0,545000.0,80.00000,360,110.0,50%-60%,Conventional:First Lien,2.0,9.0,1.0
1,2023.0,3.0,2.0,1.0,315000.0,98.97400,360,65.0,50%-60%,FHA:First Lien,3.0,9.0,4.0
2,2023.0,3.0,2.0,1.0,255000.0,96.50000,360,39.0,>60%,FHA:First Lien,2.0,10.0,1.0
3,2023.0,3.0,2.0,1.0,425000.0,90.00000,360,85.0,>60%,Conventional:First Lien,2.0,9.0,1.0
4,2023.0,3.0,2.0,32.0,425000.0,78.87900,360,158.0,30%-<36%,FHA:First Lien,1.0,9.0,1.0
5,2023.0,3.0,2.0,32.0,215000.0,84.70600,360,92.0,>60%,VA:First Lien,3.0,10.0,1.0
6,2023.0,3.0,2.0,1.0,245000.0,96.50000,360,48.0,50%-60%,FHA:First Lien,1.0,10.0,1.0
7,2023.0,3.0,2.0,32.0,565000.0,76.08700,360,134.0,50%-60%,FHA:First Lien,1.0,9.0,3.0
8,2023.0,3.0,2.0,1.0,415000.0,75.00000,360,177.0,50%-60%,Conventional:First Lien,3.0,9.0,1.0
9,2023.0,3.0,2.0,32.0,365000.0,40.78200,360,47.0,>60%,Conventional:First Lien,2.0,10.0,1.0


In [4]:
test_name = 'loan_term'
print(rejected_recovered[test_name].value_counts().reset_index())
#print(rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

int_values = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_term', 'applicant_credit_score_type',
    'co-applicant_credit_score_type', 'denial_reason-1'
]

float_values = [
    'loan_to_value_ratio', 'income', 'debt_to_income_ratio'
]

string_values = [
    'derived_loan_product_type'
]

    loan_term    count
0         360  1064391
1         300   240622
2         240   171468
3         180   113855
4         120    81959
..        ...      ...
629       441        1
630       391        1
631        21        1
632        44        1
633       456        1

[634 rows x 2 columns]


In [6]:
#Convert any ranges into the middle of that range, or leave it at that range if its too broad
test_name = 'debt_to_income_ratio'
ranges = rejected_recovered[test_name].value_counts().reset_index()

rejected_recovered[test_name] = rejected_recovered[test_name].replace(">60%", 60)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("50%-60%", 55)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("20%-<30%", (29+20)/2)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("30%-<36%", (35+30)/2)
rejected_recovered[test_name] = rejected_recovered[test_name].replace("<20%", 20)

rejected_recovered[test_name].value_counts().reset_index()

#rejected_recovered[column] = rejected_recovered[column].astype('float64', errors="ignore")

rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts().reset_index()

,debt_to_income_ratio,count
0,60,593890
1,55,316648
2,24.5,165724
3,32.5,151773
4,20,127244
5,49,49170
6,44,44828
7,42,42648
8,48,39990
9,43,38757


In [7]:
test_name = 'loan_term'
rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts().reset_index()
print(len(rejected_recovered))

2037095


In [8]:
test_name = 'income'
rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name].value_counts()

income
0.0       37693
60.0      29956
48.0      23677
50.0      23393
42.0      21579
          ...  
3566.0        1
2569.0        1
2534.0        1
1672.0        1
9741.0        1
Name: count, Length: 4604, dtype: int64

In [9]:
test_name = 'loan_to_value_ratio'
#fix any rows that have exempt

rejected_recovered[test_name] = rejected_recovered[test_name].replace('Exempt', rejected_recovered[test_name].value_counts().reset_index().iat[0, 0])

rejected_recovered[test_name]

0          80.00000
1          98.97400
2          96.50000
3          90.00000
4          78.87900
             ...   
2037090    37.44300
2037091    86.06300
2037092    23.64900
2037093    45.45500
2037094        <NA>
Name: loan_to_value_ratio, Length: 2037095, dtype: string

In [10]:
#clean up dataframe and turn it into proper typings
for column in int_values:
    print(f"changing column: {column}")
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])
    try:
        rejected_recovered[column] = rejected_recovered[column].astype('int32')
    except Exception as e:
        print(f"skipping int conversion: {column}") 
        print(f"reason: {e}")
        continue  
    

for column in float_values:
    print(f"converting float: {column}")
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])   
    try:
        rejected_recovered[column] = rejected_recovered[column].astype('float64')
    except Exception as e:
        print(f"skipping float conversion: {column}")
        print(f"reason: {e}")

for column in string_values:
    rejected_recovered[column] = rejected_recovered[column].fillna(rejected_recovered[column].value_counts().reset_index().iat[0, 0])

rejected_recovered.dtypes

changing column: activity_year
changing column: action_taken
changing column: preapproval
changing column: loan_purpose
changing column: loan_amount
changing column: loan_term
changing column: applicant_credit_score_type
changing column: co-applicant_credit_score_type
changing column: denial_reason-1
converting float: loan_to_value_ratio
converting float: income
converting float: debt_to_income_ratio


activity_year                              int32
action_taken                               int32
preapproval                                int32
loan_purpose                               int32
loan_amount                                int32
loan_to_value_ratio                      float64
loan_term                                  int32
income                                   float64
debt_to_income_ratio                     float64
derived_loan_product_type         string[python]
applicant_credit_score_type                int32
co-applicant_credit_score_type             int32
denial_reason-1                            int32
dtype: object

In [11]:

"""
for column in rejected_recovered.columns:
    if rejected_recovered[column].dtype == 'float64':
"""

rejected_recovered.isna().sum()

activity_year                     0
action_taken                      0
preapproval                       0
loan_purpose                      0
loan_amount                       0
loan_to_value_ratio               0
loan_term                         0
income                            0
debt_to_income_ratio              0
derived_loan_product_type         0
applicant_credit_score_type       0
co-applicant_credit_score_type    0
denial_reason-1                   0
dtype: int64

In [12]:
#Remove outliers from rejected df based on loan amount
rejected_recovered

"""
int_values = [
    'activity_year', 'action_taken', 'preapproval', 'loan_purpose', 'loan_amount', 'loan_term', 'applicant_credit_score_type',
    'co-applicant_credit_score_type', 'denial_reason-1'
]

float_values = [
    'loan_to_value_ratio', 'income', 'debt_to_income_ratio'
]
"""

loan_std = rejected_recovered['loan_amount'].std()
loan_mean = rejected_recovered['loan_amount'].mean()
print(loan_std)
print(loan_mean)
temp_z_data = rejected_recovered.copy()
temp_z_data['z_loan'] = ((rejected_recovered['loan_amount'] - loan_mean) / loan_std)
threshold = 3

#remove based on threshold
no_outliers_rejected = temp_z_data[(temp_z_data['z_loan'].abs()) <= threshold].drop(columns=['z_loan'])

2709981.713368594
195306.5015151478


In [13]:
no_outliers_rejected['loan_amount'].value_counts().reset_index()
print(len(no_outliers_rejected))

2036325


In [19]:
#Clean up accepted hdma loan (repeat of the rejected clean up process)
#Convert any ranges into the middle of that range, or leave it at that range if its too broad
test_name = 'debt_to_income_ratio'
ranges = accepted_recovered[test_name].value_counts().reset_index()

accepted_recovered[test_name] = accepted_recovered[test_name].replace(">60%", 60)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("50%-60%", 55)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("20%-<30%", (29+20)/2)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("30%-<36%", (35+30)/2)
accepted_recovered[test_name] = accepted_recovered[test_name].replace("<20%", 20)

accepted_recovered[test_name].value_counts().reset_index()

#accepted_recovered[column] = accepted_recovered[column].astype('float64', errors="ignore")

accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts().reset_index()

test_name = 'loan_term'
accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts().reset_index()
print(len(accepted_recovered))

test_name = 'income'
accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name].value_counts()

test_name = 'loan_to_value_ratio'
#fix any rows that have exempt

accepted_recovered[test_name] = accepted_recovered[test_name].replace('Exempt', accepted_recovered[test_name].value_counts().reset_index().iat[0, 0])

accepted_recovered[test_name]

#clean up dataframe and turn it into proper typings
for column in int_values:
    print(f"changing column: {column}")
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])
    try:
        accepted_recovered[column] = accepted_recovered[column].astype('int32')
    except Exception as e:
        print(f"skipping int conversion: {column}") 
        print(f"reason: {e}")
        continue  
    

for column in float_values:
    print(f"converting float: {column}")
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])   
    try:
        accepted_recovered[column] = accepted_recovered[column].astype('float64')
    except Exception as e:
        print(f"skipping float conversion: {column}")
        print(f"reason: {e}")

for column in string_values:
    accepted_recovered[column] = accepted_recovered[column].fillna(accepted_recovered[column].value_counts().reset_index().iat[0, 0])

loan_std = accepted_recovered['loan_amount'].std()
loan_mean = accepted_recovered['loan_amount'].mean()
print(loan_std)
print(loan_mean)
temp_z_data = accepted_recovered.copy()
temp_z_data['z_loan'] = ((accepted_recovered['loan_amount'] - loan_mean) / loan_std)
threshold = 3

#remove based on threshold
no_outliers_accepted = temp_z_data[(temp_z_data['z_loan'].abs()) <= threshold].drop(columns=['z_loan'])

3452631
changing column: activity_year
changing column: action_taken
changing column: preapproval
changing column: loan_purpose
changing column: loan_amount
changing column: loan_term
changing column: applicant_credit_score_type
changing column: co-applicant_credit_score_type
changing column: denial_reason-1
converting float: loan_to_value_ratio
converting float: income
converting float: debt_to_income_ratio
852443.5692311071
371280.80151050026


In [20]:
#delete_large_files()

no_outliers_accepted.head(10)
no_outliers_accepted.isna().sum()

activity_year                     0
action_taken                      0
preapproval                       0
loan_purpose                      0
loan_amount                       0
loan_to_value_ratio               0
loan_term                         0
income                            0
debt_to_income_ratio              0
derived_loan_product_type         0
applicant_credit_score_type       0
co-applicant_credit_score_type    0
denial_reason-1                   0
dtype: int64